In [ ]:
# =========================================================
# IMAGE CLASSIFICATION USING CNN (TENSORFLOW)
# DATA + MODEL + TRAINING + TEST DATA + EVALUATION
# =========================================================

# -------------------------------
# 1. IMPORT LIBRARIES
# -------------------------------
!pip install tensorflow

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2

print("TensorFlow Version:", tf.__version__)


# -------------------------------
# 2. LOAD DATASET (DATA GIVEN)
# -------------------------------
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("Training Images:", x_train.shape)
print("Test Images:", x_test.shape)


# -------------------------------
# 3. PREPROCESS DATA
# -------------------------------
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)


# -------------------------------
# 4. CLASS NAMES
# -------------------------------
class_names = [
    'airplane','automobile','bird','cat','deer',
    'dog','frog','horse','ship','truck'
]


# -------------------------------
# 5. BUILD CNN MODEL
# -------------------------------
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Conv2D(128, (3,3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.summary()


# -------------------------------
# 6. COMPILE MODEL
# -------------------------------
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# -------------------------------
# 7. TRAIN MODEL
# -------------------------------
history = model.fit(
    x_train,
    y_train,
    epochs=10,
    batch_size=64,
    validation_split=0.2
)


# -------------------------------
# 8. EVALUATE ON TEST DATASET
# -------------------------------
test_loss, test_accuracy = model.evaluate(x_test, y_test)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


# -------------------------------
# 9. PLOT ACCURACY & LOSS
# -------------------------------
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title("Accuracy")

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title("Loss")

plt.show()


# -------------------------------
# 10. PREDICTION ON TEST IMAGE
# -------------------------------
predictions = model.predict(x_test)

index = 0
plt.imshow(x_test[index])
plt.title("Predicted: " + class_names[np.argmax(predictions[index])])
plt.axis('off')
plt.show()


# -------------------------------
# 11. SAVE TEST DATA AS IMAGES
# -------------------------------
save_dir = "test_dataset"
os.makedirs(save_dir, exist_ok=True)

(_, _), (x_raw_test, y_raw_test) = cifar10.load_data()

for i in range(100):   # saving first 100 test images
    label = class_names[y_raw_test[i][0]]
    class_dir = os.path.join(save_dir, label)
    os.makedirs(class_dir, exist_ok=True)

    img = cv2.cvtColor(x_raw_test[i], cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"{class_dir}/img_{i}.png", img)

print("Test dataset images saved successfully!")


# -------------------------------
# 12. SAVE TRAINED MODEL
# -------------------------------
model.save("cnn_image_classification_model.h5")
print("Model saved successfully!")
